In [ ]:
import joblib
import pandas as pd
import numpy as np


# Andre Trænings perioder

In [ ]:
! pip install catboost
import numpy as np
import pandas as pd

## Funktioner

In [ ]:
def ml_pipeline (fil,train_start,val_end,test_start,test_end):
  # importing df and basic cleaning
  df_ml = pd.read_csv(fil)
  df_ml["Forecasttidspunkt"] = pd.to_datetime(df_ml["Forecasttidspunkt"])
  df_ml["periode_regnskabstal"] = pd.to_datetime(df_ml["periode_regnskabstal"])
  df_ml.set_index("Forecasttidspunkt",inplace=True)


  #Dropping non relevant columns
  df_ml.rename(columns={"CUSIP":"cusip"},inplace=True)
  list_drop = ["SHROUT","company_name","cusip","analyst_EPS_mean","analyst_high","analyst_low","analyst_std_mean","periode_regnskabstal","PERMNO","date","slutmåned","Instrument"]
  df_ml.drop(list_drop,axis=1,inplace=True,errors="ignore")

  # Creating year and month
  df_ml["month"] = df_ml.index.month
  df_ml["year"] = df_ml.index.year
  df_ml = df_ml.astype({'GVKEY': 'category'})
  df_ml = df_ml.astype({'industry_fama': 'category'})
  df_ml.sort_index(inplace=True)
  
  # New features og duplikat kolonner
  df_ml["volume_usd"] = df_ml["ALTPRC"] * df_ml["VOL"]
  df_ml["Div_yield"] = df_ml["DPS_ex_date(dvpsxq)"] / df_ml["ALTPRC"]
  df_ml.drop(["PRC","adj_close"],axis=1,inplace=True)

  columns_drop_list = ["Assets_total(Atq)","industry_return","VOL","volume_usd"]
  df_ml.drop(columns_drop_list,axis=1,inplace=True)


  # Lagging variable

  columns_lag = ['EPS_actual', 'Accounts_payable(Apq)',
       'Assets_total(Atq)', 'COGS(Cogsq)',
       'Common_equity(Ceqq)', 'DPS_ex_date(dvpsxq)', 'Deprect_and_ammor(Dpq)',
       'EPS_lagged', 'GVKEY', 'Income taxes_total(Txtq)',
       'Non_operating_income(Nopiq)', 'Operating_income_after_deprec(Oiadpq)',
       'Opex_total(Xoprq)', 'PP&E_total_net(Ppentq)','PRC',
       'Pretax_income(Piq)', 'Receviables(Rectq)', 'Revenue_total(Revtq)',
    'VOL', 'adj_return','adj_close','market_cap', 'niq', 'industry_return', 'merafkast', 'ALTPRC','volume_usd','Div_yield']

  df_lag = df_ml.copy()
  df_lag = df_lag[df_lag.columns.intersection(columns_lag)]
  df_lag_new = df_lag.copy()

  for column in df_lag.columns:
    for i in range(1,6):
      df_lag_new[f'{column}_{i}'] = df_lag.groupby("GVKEY")[f'{column}'].shift(i).values

  columns_to_choose = ['Assets_total(Atq)','Accounts_payable(Apq)', 'COGS(Cogsq)',
       'Common_equity(Ceqq)', 'DPS_ex_date(dvpsxq)', 'Deprect_and_ammor(Dpq)',
       'EPS_lagged', 'Income taxes_total(Txtq)',
       'Non_operating_income(Nopiq)', 'Operating_income_after_deprec(Oiadpq)',
       'Opex_total(Xoprq)', 'PP&E_total_net(Ppentq)',
       'Pretax_income(Piq)', 'Receviables(Rectq)', 'Revenue_total(Revtq)',
       'market_cap', 'niq',"Div_yield","ALTPRC",'adj_return',"industry_return","merafkast",'VOL','volume_usd','PRC','adj_close']
  col_lag_choose = []
  for element in columns_to_choose:
    for i in range(1,6):
      col_lag_choose.append(f'{element}_{i}')
  col_lag_choose.append("GVKEY")

  # Vælger kun de endelige features
  df_lag_new = df_lag_new[df_lag_new.columns.intersection(col_lag_choose)]

  # Merge lags og oprindelig dataframe sammen
  df_ml = df_ml.reset_index().merge(df_lag_new.reset_index(),how="inner",on=["GVKEY","Forecasttidspunkt"])

  # Fikse indeks og erstatte NA i lags med NA

  numeric_columns = df_ml.select_dtypes(include=['number']).columns
  df_ml[numeric_columns] = df_ml[numeric_columns].fillna(0)

  df_ml.set_index("Forecasttidspunkt",inplace=True)
  

  cutoff_date = "2016-06-30"
  validation_start = "2016-09-30"
  validation_end = "2018-06-30"

  training_df = df_ml.loc[:cutoff_date,:]
  validation_df = df_ml.loc[validation_start:validation_end,:]
  test_df = df_ml.loc[test_start:test_end,:]

  training_df_cv =  df_ml.loc[train_start:val_end,:]

# X og Y

  training_y = training_df["EPS_actual"]
  validatation_y = validation_df["EPS_actual"]
  test_y = test_df["EPS_actual"]
  training_y_cv = training_df_cv["EPS_actual"]

  training_x = training_df.drop("EPS_actual",axis=1)
  training_x_cv = training_df_cv.drop("EPS_actual",axis=1)
  validatation_x = validation_df.drop("EPS_actual",axis=1)
  test_x = test_df.drop("EPS_actual",axis=1)


  from catboost import CatBoostRegressor
  from catboost import Pool

  params = {'boosting_type': 'Plain','bootstrap_type': 'MVS',
 'colsample_bylevel': 0.09413483344915186,'depth': 8,
 'iterations': 1000,
 "loss_function": "MAE",
  "max_ctr_complexity": 0,
  "one_hot_max_size": 5000,
  "cat_features": ["GVKEY","industry_fama"]}

  cat_model = CatBoostRegressor(**params,random_seed=2021)


  cat_model.fit(training_x_cv,training_y_cv,verbose=False)

  print(cat_model.best_iteration_)
  print(cat_model.get_best_score())

  test_pred = cat_model.predict(test_x)

  return test_pred

In [ ]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
import numpy as np


  # Metric Dataframe
def metric_dataframe (test_predicted,train_start,train_end,test_start,test_end, fil="/content/drive/MyDrive/Cand.fælles/Speciale/datarobot_input/input.csv.zip"):

  '''fil : Indsæt stien til den fil der indeholder data før ML rensning
    test_predicted : Henvis til et array af predicted værdier på testsættet'''

  df = pd.read_csv(fil)
  df["Forecasttidspunkt"] = pd.to_datetime(df["Forecasttidspunkt"])
  df.set_index("Forecasttidspunkt",inplace=True)
  df.sort_index(inplace=True)
  train_df = df.loc[train_start:train_end,:]
  test_df = df.loc[test_start:test_end,:]
  test_df["Model_predicted"] = test_predicted
  test_df.dropna(subset=["analyst_EPS_mean"],inplace=True)

 
  return test_df

  # Metric Calculations
def metric_calculations (metric_test_df):
  # Mae
  analytiker_mae = mean_absolute_error(metric_test_df["EPS_actual"],metric_test_df["analyst_EPS_mean"])
  model_mae = mean_absolute_error(metric_test_df["EPS_actual"],metric_test_df["Model_predicted"])
  model_anal = analytiker_mae - model_mae
  model_out_pct = -(model_mae/analytiker_mae -1)


  metric_df = pd.DataFrame.from_dict({"MAE":[analytiker_mae,model_mae,model_anal,model_out_pct]})
  metric_df.index = ["Analytiker","Model","Forskel","Forskel(%)"]
  return metric_df

In [ ]:
df_1 = joblib.load("/content/drive/MyDrive/Cand.fælles/Speciale/Python/Q4_test_base.pkl")

In [ ]:
df_1

12-31

03-31

06-30

09-30

In [ ]:
train_start = ["1998-06-30","2003-06-30", "2005-06-30", "2007-06-30", "2009-06-30","2011-06-30"]
train_end =  ["2006-06-30","2008-09-30","2010-09-30","2012-09-30","2014-09-30","2016-09-30"]
test_start = ["2006-09-30","2008-12-31","2010-12-31","2012-12-31","2014-12-31","2016-12-31"] 
test_end = ["2008-09-30","2010-09-30","2012-09-30","2014-09-30","2016-09-30","2018-09-30"]

fil_sti = "/content/drive/MyDrive/Cand.fælles/Speciale/datarobot_input/input.csv.zip"

Q1_performance = {}
Q1_test = {}

for dato in zip(train_start,train_end,test_start,test_end):
  Q1_test[f'{dato[2]}'] = metric_dataframe(test_predicted=(ml_pipeline(fil=fil_sti,train_start=f'{dato[0]}',val_end=f'{dato[1]}',test_start=f'{dato[2]}',test_end=f'{dato[3]}')),fil=fil_sti,train_start=f'{dato[0]}',train_end=f'{dato[1]}',test_start=f'{dato[2]}',test_end=f'{dato[3]}')
  Q1_performance[f'{dato[2]}'] = metric_calculations((Q1_test[f'{dato[2]}']))


In [ ]:
pd.concat(Q1_performance)

### Q2

In [ ]:
train_start = ["1998-06-30","2003-06-30", "2005-06-30", "2007-06-30", "2009-06-30","2011-06-30"]
train_end =  ["2006-06-30","2008-09-30","2010-09-30","2012-09-30","2014-09-30","2016-09-30"]
test_start = ["2006-09-30","2008-12-31","2010-12-31","2012-12-31","2014-12-31","2016-12-31"] 
test_end = ["2008-09-30","2010-09-30","2012-09-30","2014-09-30","2016-09-30","2018-09-30"]

fil_sti = "/content/drive/MyDrive/Cand.fælles/Speciale/datarobot_input/input_2Q.csv.zip"

Q2_performance = {}
Q2_test = {}

for dato in zip(train_start,train_end,test_start,test_end):
  Q2_test[f'{dato[2]}'] = metric_dataframe(test_predicted=(ml_pipeline(fil=fil_sti,train_start=f'{dato[0]}',val_end=f'{dato[1]}',test_start=f'{dato[2]}',test_end=f'{dato[3]}')),fil=fil_sti,train_start=f'{dato[0]}',train_end=f'{dato[1]}',test_start=f'{dato[2]}',test_end=f'{dato[3]}')
  Q2_performance[f'{dato[2]}'] = metric_calculations((Q2_test[f'{dato[2]}']))

In [ ]:
pd.concat(Q2_performance)

### Q4

In [ ]:
train_start = ["1998-06-30","2003-06-30", "2005-06-30", "2007-06-30", "2009-06-30","2011-06-30"]
train_end =  ["2006-06-30","2008-09-30","2010-09-30","2012-09-30","2014-09-30","2016-09-30"]
test_start = ["2006-09-30","2008-12-31","2010-12-31","2012-12-31","2014-12-31","2016-12-31"] 
test_end = ["2008-09-30","2010-09-30","2012-09-30","2014-09-30","2016-09-30","2018-09-30"]

fil_sti = "/content/drive/MyDrive/Cand.fælles/Speciale/datarobot_input/input_4Q.csv.zip"

Q4_performance = {}

Q4_test = {}

for dato in zip(train_start,train_end,test_start,test_end):
  Q4_test[f'{dato[2]}'] = metric_dataframe(test_predicted=(ml_pipeline(fil=fil_sti,train_start=f'{dato[0]}',val_end=f'{dato[1]}',test_start=f'{dato[2]}',test_end=f'{dato[3]}')),fil=fil_sti,train_start=f'{dato[0]}',train_end=f'{dato[1]}',test_start=f'{dato[2]}',test_end=f'{dato[3]}')
  Q4_performance[f'{dato[2]}'] = metric_calculations((Q4_test[f'{dato[2]}']))

In [ ]:
pd.concat(Q4_performance)

In [ ]:
Q4_test["2014-12-31"]

In [ ]:
samlet_df = pd.concat([pd.concat(Q1_performance),pd.concat(Q2_performance),pd.concat(Q4_performance)],axis=1)

In [ ]:
samlet_df.columns = ["Q1","Q2","Q4"]
samlet_df

In [ ]:
joblib.dump(samlet_df,"/content/drive/MyDrive/Cand.fælles/Speciale/Python/robusthed/samlet_df_performance_rolling.pkl")

In [ ]:
samlet_df.round(2)